In [1]:
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
from keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
mp_face_detect = mp.solutions.face_detection
face_detect = mp_face_detect.FaceDetection (
    min_detection_confidence=0.6
)
model_file_path = '../models/best-final/VGG16-best_val_loss_model.h5'
label_classes = {
    0: 'angry', 
    1: 'contempt', 
    2: 'disgust', 
    3: 'fear', 
    4: 'happy', 
    5: 'neutral', 
    6: 'sad', 
    7: 'surprise'
}
cam_src_id = 0
primary_color = (0, 0, 255)
secondary_color = (255, 255, 255)

In [3]:
def load_model(model_file_path=model_file_path):
    model = Sequential()

    model.add(Conv2D(64, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block1_conv1', input_shape=(48, 48, 1) ))
    model.add(Conv2D(64, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block1_conv2'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block1_maxpool'))

    model.add(Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block2_conv1'))
    model.add(Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block2_conv2'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block2_maxpool'))

    model.add(Conv2D(256, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block3_conv1'))
    model.add(Conv2D(256, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block3_conv2'))
    model.add(Conv2D(256, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block3_conv3'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block3_maxpool'))

    model.add(Conv2D(512, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block4_conv1'))
    model.add(Conv2D(512, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block4_conv2'))
    model.add(Conv2D(512, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block4_conv3'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block4_maxpool'))

    model.add(Conv2D(512, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block5_conv1'))
    model.add(Conv2D(512, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block5_conv2'))
    model.add(Conv2D(512, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block5_conv3'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block5_maxpool'))

    model.add(Flatten())
    model.add(Dense(4096, kernel_regularizer=l2(1e-7), activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(4096, kernel_regularizer=l2(1e-7), activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(8, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=0.0002), loss='categorical_crossentropy', metrics=['accuracy']) 
    model.load_weights(model_file_path)

    return model

In [4]:
def predict(model, res=(48, 48), image=None, label_classes=label_classes, num_preds=3):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (res[0], res[1]))
    image = image / 255.0
    image = np.reshape(image, (1, image.shape[0], image.shape[1], 1))
    preds = model.predict(image)
    sorted_preds = np.argsort(preds)
    best_pred_indexes = sorted_preds[0][-1: -num_preds - 1: -1]
    result = {}
    
    for i, p in enumerate(best_pred_indexes):
        result[i + 1] = {'class': label_classes[p], 'acc': int(preds[0][p] * 100)}
    
    return result

In [7]:
model = load_model()
cam_src = cv2.VideoCapture('../resources/test_captures/06.mp4') 

while True:
    ret, frame = cam_src.read()
    
    if ret:
        try:
            frame = cv2.resize(frame, (500, 500))
            height, width, _ = frame.shape
            rgb_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_detect_results = face_detect.process(rgb_img)
            
            if face_detect_results.detections:
                for face in face_detect_results.detections:
                    rel_bound_rect = face.location_data.relative_bounding_box
                    face_x, face_w, face_y, face_h = int(rel_bound_rect.xmin * width), int(rel_bound_rect.width * width), int(rel_bound_rect.ymin * height), int(rel_bound_rect.height * height)
                    face = frame[face_y: face_y + face_h, face_x: face_x + face_w]
                    best_3_preds = predict(model=model, image=face)
                    pred_progress_fill_1 = int(round((best_3_preds[1]['acc'] / 100) * ((face_x + face_w) - (face_x + 85)), 0))
                    pred_progress_fill_2 = int(round((best_3_preds[2]['acc'] / 100) * ((face_x + face_w) - (face_x + 85)), 0))
                    pred_progress_fill_3 = int(round((best_3_preds[3]['acc'] / 100) * ((face_x + face_w) - (face_x + 85)), 0))
                    
                    cv2.rectangle(frame, (face_x, face_y - 65), (face_x + face_w + 1, face_y - 4), (0, 0, 0), -1)
                    cv2.putText(frame, f"{best_3_preds[1]['class']}", (face_x, face_y - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.55, secondary_color, 2)
                    cv2.putText(frame, f"{best_3_preds[2]['class']}", (face_x, face_y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.55, secondary_color, 2)
                    cv2.putText(frame, f"{best_3_preds[3]['class']}", (face_x, face_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.55, secondary_color, 2)
                    cv2.rectangle(frame, (face_x + 85, face_y - 60), (face_x + face_w, face_y - 50), secondary_color, -1)
                    cv2.rectangle(frame, (face_x + 85, face_y - 60), (face_x + 85 + pred_progress_fill_1, face_y - 50), primary_color, -1)
                    cv2.rectangle(frame, (face_x + 85, face_y - 40), (face_x + face_w, face_y - 30), secondary_color, -1)
                    cv2.rectangle(frame, (face_x + 85, face_y - 40), (face_x + 85 + pred_progress_fill_2, face_y - 30), (0, 255, 0), -1)
                    cv2.rectangle(frame, (face_x + 85, face_y - 20), (face_x + face_w, face_y - 10), secondary_color, -1)
                    cv2.rectangle(frame, (face_x + 85, face_y - 20), (face_x + 85 + pred_progress_fill_3, face_y - 10), (255, 0, 0), -1)
                    cv2.rectangle(frame, (face_x, face_y), (face_x + face_w, face_y + face_h), primary_color, 2)
                    
            cv2.imshow('[LIVE] VGG16 Facial Expression Recognition', frame)
            
        except:
            pass
    else:
        break

    k = cv2.waitKey(1)
    if k == 27:
        break
    
cam_src.release()
cv2.destroyAllWindows()

In [8]:
# image = cv2.imread('test-5.jpg')
# image = cv2.resize(image, (600, 500))
# height, width, _ = image.shape
# rgb_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# face_detect_results = face_detect.process(rgb_img)

# if face_detect_results.detections:
#     for face in face_detect_results.detections:
#         rel_bound_rect = face.location_data.relative_bounding_box
#         face_x, face_w, face_y, face_h = int(rel_bound_rect.xmin * width), int(rel_bound_rect.width * width), int(rel_bound_rect.ymin * height), int(rel_bound_rect.height * height)
#         face = image[face_y: face_y + face_h, face_x: face_x + face_w]
#         best_3_preds = predict(model=model, image=face)
#         pred_progress_fill_1 = int(round((best_3_preds[1]['acc'] / 100) * ((face_x + face_w) - (face_x + 85)), 0))
#         pred_progress_fill_2 = int(round((best_3_preds[2]['acc'] / 100) * ((face_x + face_w) - (face_x + 85)), 0))
#         pred_progress_fill_3 = int(round((best_3_preds[3]['acc'] / 100) * ((face_x + face_w) - (face_x + 85)), 0))

#         cv2.putText(image, f"{best_3_preds[1]['class']}", (face_x, face_y - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.55, secondary_color, 2)
#         cv2.putText(image, f"{best_3_preds[2]['class']}", (face_x, face_y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.55, secondary_color, 2)
#         cv2.putText(image, f"{best_3_preds[3]['class']}", (face_x, face_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.55, secondary_color, 2)
#         cv2.rectangle(image, (face_x + 85, face_y - 60), (face_x + face_w, face_y - 50), secondary_color, -1)
#         cv2.rectangle(image, (face_x + 85, face_y - 60), (face_x + face_w, face_y - 50), (0, 0, 0), 2)
#         cv2.rectangle(image, (face_x + 85, face_y - 60), (face_x + 85 + pred_progress_fill_1, face_y - 50), primary_color, -1)

#         cv2.rectangle(image, (face_x + 85, face_y - 40), (face_x + face_w, face_y - 30), secondary_color, -1)
#         cv2.rectangle(image, (face_x + 85, face_y - 40), (face_x + face_w, face_y - 30), (0, 0, 0), 2)
#         cv2.rectangle(image, (face_x + 85, face_y - 40), (face_x + 85 + pred_progress_fill_2, face_y - 30), (0, 255, 0), 2)

#         cv2.rectangle(image, (face_x + 85, face_y - 20), (face_x + face_w, face_y - 10), secondary_color, -1)
#         cv2.rectangle(image, (face_x + 85, face_y - 20), (face_x + face_w, face_y - 10), (0, 0, 0), 2)
#         cv2.rectangle(image, (face_x + 85, face_y - 20), (face_x + 85 + pred_progress_fill_3, face_y - 10), (255, 0, 0), 2)

#         cv2.rectangle(image, (face_x, face_y), (face_x + face_w, face_y + face_h), primary_color, 2)

# cv2.imshow('LIVE', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
